First, we need some enum classes

In [18]:
# define the alignment of properties of the Enum classes
from enum import Enum

# define Alignment
class AttributeAlignment(Enum):
    ONE_LINE = "one_line"
    MULTI_LINE = "multi_line"

# define Indentations
class Indentation(Enum):
    TWO_SPACES = "  "
    FOUR_SPACES = "    "
    TAB = "\t"

# define selfclosing
class SelfClosing(Enum):
    CLOSE = "close"
    OPEN = "open"
    KEEP_ORIGINAL = "keep_original"


Next is the Xml class, which is used to parse and construct XML:

In [19]:
import re
from typing import Dict, List, Union

class Xml:
    def __init__(self, node_name: str = "", attributes: Dict[str, str] = None, content: str = None):
        self.node_name = node_name  # Node name
        self.children = []  # List of child nodes
        self.attributes = attributes if attributes else {}  # Attribute dictionary
        self.str = content if content else ""  # Node content
        self.self_closed = False  # Is it a self-closing tag
        
        if node_name and not content:  # If a node name is provided but no content, parse it
            self.parse(node_name)

    def parse(self, input_xml: str):
        # Use regular expressions to parse the XML string
        pattern = re.compile(r'<(.*?)( .*?)?>(.*)</\1>', re.DOTALL)
        matcher = pattern.match(input_xml)
        if matcher:
            self.node_name = matcher.group(1)  # Get the node name
            attributes_str = matcher.group(2)  # Get the attribute string
            self.match_node_attributes(attributes_str)  # Match node attributes
            node_content = matcher.group(3)  # Get the node content
            self.match_node_list(node_content)  # Match the list of child nodes

    def match_node_attributes(self, attributes_str: str):
        if not attributes_str:  # If the attribute string is empty, return directly
            return
        attributes = {}  # Create attribute dictionary
        attributes_arr = attributes_str.strip().split(" ")  # Split the attribute string into a list
        for attribute in attributes_arr:
            key, value = attribute.split("=")  # Split each attribute into key-value pairs
            attributes[key] = value.strip('"')  # Remove the quotes around the value and add to the dictionary
        self.attributes = attributes  # Update the node's attribute dictionary

    def match_node_list(self, node_content: str):
        if not node_content:  # If the node content is empty, return directly
            return
        # Match complete nodes
        pattern_full = re.compile(r'<(.*?)( .*?)?>(.*)</\1>', re.DOTALL)
        # Match self-closing nodes
        pattern_simple = re.compile(r'<([^>]*?)( .*?)?(\\s*)/>', re.DOTALL)
        while True:
            matcher_full = pattern_full.match(node_content)
            matcher_simple = pattern_simple.match(node_content)
            if matcher_full:
                current_matcher = matcher_full  # Current matcher is for complete nodes
                matched_simple = False
            elif matcher_simple:
                current_matcher = matcher_simple  # Current matcher is for self-closing nodes
                matched_simple = True
            else:
                break
            
            child_node = Xml()  # Create child node
            child_node.self_closed = matched_simple  # Set whether the child node is self-closing
            child_node.node_name = current_matcher.group(1)  # Get the child node name
            attributes_str = current_matcher.group(2)  # Get the child node's attribute string
            child_node.match_node_attributes(attributes_str)  # Match the child node's attributes
            child_node_content = current_matcher.group(3) if not matched_simple else ""  # Get the child node content
            if not matched_simple:
                child_node.match_node_list(child_node_content)  # Recursively match the list of child nodes
            self.children.append(child_node)  # Add the child node to the list of child nodes
        self.str = node_content  # Update the node content

    def add_node(self, new_node_name: str, new_node: Union[Dict[str, str], str]):
        if isinstance(new_node, dict):  # If the new node is a dictionary
            for k, v in new_node.items():
                child = Xml(k, content=v)  # Create child node
                self.children.append(child)  # Add the child node to the list of child nodes
        elif isinstance(new_node, str):  # If the new node is a string
            child = Xml(new_node_name, content=new_node)  # Create child node
            self.children.append(child)  # Add the child node to the list of child nodes

    def print(self, indentation: Indentation, self_closing: SelfClosing, attribute_alignment: AttributeAlignment, one_line_string_node: bool) -> str:
        # Print the XML content
        return self.print_internal(self, "", indentation, 0, self_closing, attribute_alignment, one_line_string_node)

    def print_internal(self, node, result: str, indentation: Indentation, current_indentation_count: int, self_closing: SelfClosing, attribute_alignment: AttributeAlignment, one_line_string_node: bool) -> str:
        indentation_str = indentation.value * current_indentation_count  # Calculate indentation string
        empty_node = (not node.children and not node.str)  # Determine if the node is empty
        str_node = (not node.children and node.str)  # Determine if the node is a string node
        result += self.append_opening_part(indentation_str, self_closing, node, empty_node, attribute_alignment, one_line_string_node)
        if str_node:
            result += node.str  # Add the node content
        else:
            for child in node.children:
                result += self.print_internal(child, "", indentation, current_indentation_count + 1, self_closing, attribute_alignment, one_line_string_node)
        result += self.append_closing_part(indentation_str, self_closing, node, empty_node)
        return result

    def append_opening_part(self, indentation_str, self_closing, node, empty_node, attribute_alignment, one_line_string_node):
        result = f"{indentation_str}<{node.node_name}"  # Add opening tag
        result += self.append_attributes(node.attributes, attribute_alignment)  # Add attributes
        if self_closing == SelfClosing.OPEN or (not node.self_closed and self_closing == SelfClosing.KEEP_ORIGINAL):
            result += ">"
        if empty_node and (self_closing == SelfClosing.CLOSE or node.self_closed and self_closing == SelfClosing.KEEP_ORIGINAL):
            result += " />"
        return result

    def append_closing_part(self, indentation_str, self_closing, node, empty_node):
        if not empty_node:
            return f"{indentation_str}</{node.node_name}>"
        return ""

    def append_attributes(self, attributes, attribute_alignment):
        result = ""
        for key, value in attributes.items():
            if attribute_alignment == AttributeAlignment.ONE_LINE:
                result += f' {key}="{value}"'
            else:
                result += f'\n  {key}="{value}"'
        return result


Finally, define a main function to test and demonstrate these features:

In [20]:
def main():
    # Create the root node and add child nodes
    xml = Xml("<root></root>")
    xml.add_node("name", {"sss": "aaa"})
    print(xml.print(Indentation.FOUR_SPACES, SelfClosing.OPEN, AttributeAlignment.ONE_LINE, True))
    print()

    # Parse and print an XML string
    xml1 = Xml("""
        <note>
          <to>Tove</to>
          <from>Junyi</from>
          <heading>Reminder</heading>
          <body><p>I want you to remember <bold>me</bold> this weekend!</p></body>
        </note>""")
    print("one line:")
    print(xml1.print(Indentation.FOUR_SPACES, SelfClosing.OPEN, AttributeAlignment.ONE_LINE, True))
    print()

    # Parse and print an XML string containing self-closing tags
    xml2 = Xml("""
        <note>
          <to>Henry</to>
          <from>Junyi</from>
          <heading>Reminder</heading>
          <sub />
          <body><p>I want you to remember <bold>me</bold> this weekend!</p></body>
        </note>""")
    print("one line with self-closed node, open it when print:")
    print(xml2.print(Indentation.FOUR_SPACES, SelfClosing.OPEN, AttributeAlignment.ONE_LINE, True))
    print()

    # Parse and print an XML string containing attributes
    xml3 = Xml("""
        <note date="2024-01-01" weather="sunny">
          <to>Tove</to>
          <from>Junyi</from>
          <heading>Reminder</heading>
          <sub property="good"/>
          <sub1 />
          <time timezone="UTC">noon</time>
          <body><p>I want you to remember <bold>me</bold> this weekend!</p></body>
        </note>""")
    print("attributes one line:")
    print(xml3.print(Indentation.TAB, SelfClosing.KEEP_ORIGINAL, AttributeAlignment.ONE_LINE, True))
    print()

    print("attributes multi-line:")
    print(xml3.print(Indentation.TAB, SelfClosing.KEEP_ORIGINAL, AttributeAlignment.MULTI_LINE, False))
    print()

if __name__ == "__main__":
    main()


<root>    <sss>aaa    </sss></root>

one line:
<
        <note>
          <to>Tove</to>
          <from>Junyi</from>
          <heading>Reminder</heading>
          <body><p>I want you to remember <bold>me</bold> this weekend!</p></body>
        </note>>

one line with self-closed node, open it when print:
<
        <note>
          <to>Henry</to>
          <from>Junyi</from>
          <heading>Reminder</heading>
          <sub />
          <body><p>I want you to remember <bold>me</bold> this weekend!</p></body>
        </note>>

attributes one line:
<
        <note date="2024-01-01" weather="sunny">
          <to>Tove</to>
          <from>Junyi</from>
          <heading>Reminder</heading>
          <sub property="good"/>
          <sub1 />
          <time timezone="UTC">noon</time>
          <body><p>I want you to remember <bold>me</bold> this weekend!</p></body>
        </note>>

attributes multi-line:
<
        <note date="2024-01-01" weather="sunny">
          <to>Tove</to>
       